In [22]:
import numpy as np
import os
import cv2
import pickle
import matplotlib.pyplot as plt
% matplotlib inline

In [23]:
def mask_overlap(mask11, mask22):
###############################################
###############################################
    """
    This function calculate region IOU when masks are
    inside different boxes
    Returns:
        intersection over unions of this two masks
    """
    mask1 = mask11.copy()
    mask2 = mask22.copy()
    
    mask1[mask1>=1] = 1
    mask2[mask2>=1] = 1
    
    #plt.subplot(1,2,1) 
    #plt.imshow(mask1)
    #plt.subplot(1,2,2) 
    #plt.imshow(mask2)
    
    green = (255,255,255)
    #box1 / box2 are the bounding box of mask1 / mask2 
    blob1 = np.where(mask1>=1)
    box1x1 = min(blob1[1])
    box1x2 = max(blob1[1])
    box1y1 = min(blob1[0])
    box1y2 = max(blob1[0])
    
    #cv2.rectangle(mask1,(box1x1,box1y1),(box1x2,box1y2),0.5,1)
    
    #plt.subplot(1,2,1) 
    #plt.imshow(mask1)
    #plt.subplot(1,2,2) 
    #plt.imshow(mask2)
    
    #print box1x1,box1x2,box1y1,box1y2
    
    blob2 = np.where(mask2>=1)
    box2x1 = min(blob2[1])
    box2x2 = max(blob2[1])
    box2y1 = min(blob2[0])
    box2y2 = max(blob2[0])
    
    #cv2.rectangle(mask2,(box2x1,box2y1),(box2x2,box2y2),0.5,1)
    
    x1 = max(box1x1 , box2x1)
    y1 = max(box1y1 , box2y1)
    x2 = min(box1x2 , box2x2)
    y2 = min(box1y2 , box2y2)

    #######################
    
    if x1 > x2 or y1 > y2:
        return 0
    w = x2 - x1 + 1
    h = y2 - y1 + 1
    # get masks in the intersection part
    inter_maska = mask1[y1: y1 + h, x1:x1 + w]
    inter_maskb = mask2[y1: y1 + h, x1:x1 + w]

    #print inter_maska
    #print inter_maskb
    
    #plt.subplot(2,2,1) 
    #plt.imshow(inter_maska)
    #plt.subplot(2,2,2) 
    #plt.imshow(inter_maskb)    
    
    assert inter_maska.shape == inter_maskb.shape
    inter = np.logical_and(inter_maskb, inter_maska).sum()
    #print inter_maskb
    #print inter_maska
    union = mask1.sum() + mask2.sum() - inter
    
    if union < 1.0:
        return 0
    return float(inter) / float(union)


In [24]:
def voc_ap(rec, prec, use_07_metric=False):
    """
    average precision calculations
    [precision integrated to recall]
    :param rec: recall
    :param prec: precision
    :param use_07_metric: 2007 metric is 11-recall-point based AP
    :return: average precision
    """
    if use_07_metric:
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap += p / 11.
    else:
        # append sentinel values at both ends
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute precision integration ladder
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # look for recall value changes
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # sum (\delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [25]:
def ClassifyColor(Red,Green,Blue):
    if Blue == 255:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            return 'CubeBody'
        elif Red % 10 == 0 and Green == Red and Red != 0:
            return 'CubeFace'
    elif Blue == 200:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            return 'CylinderBody'
        elif Red % 10 == 0 and Green == Red and Red != 0:
            return 'CylinderFace'
    elif Blue == 150:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            #return 'Handle'
            return 'CylinderBody'
    else:
        if Red == 0 and Green == 0 and Blue == 0:
            return 'Background'
    return 'Invalid'

In [26]:
def ParseMask(img,h,w):
    part = []
    partlabel = []
    visited_color = []
    
    blue_channel = img[:,:,0]
    pixel = blue_channel.nonzero()
    for i in range(0,len(pixel[0])):
        color = img[pixel[0][i],pixel[1][i]]
        color_string =  str("%03d" % color[0]) + str("%03d" % color[1]) +str("%03d" % color[2])
        if color_string not in visited_color: 
            visited_color.append(color_string)
            label = ClassifyColor(color[2],color[1],color[0])
            if label!= 'Background' and label!='Invalid':
                lower = np.array(color, dtype = "uint8")#颜色下限  
                upper = np.array(color, dtype = "uint8")#颜色上限  

                #plt.imshow(fcis_img)
                mask = cv2.inRange(img, lower, upper) 
                output = cv2.bitwise_and(img,img,mask=mask)
                output = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
                output[output>=1] = 1

                part.append(output)
                partlabel.append(label)
    
    return part, partlabel

In [27]:
import os
from glob import glob 
def search_file(pattern, search_path=os.environ['PATH'], pathsep=os.pathsep):
    for path in search_path.split(os.pathsep):
        for match in glob(os.path.join(path, pattern)):
            yield match

In [28]:
def mask_overlap_box(box1, box2, mask1, mask2):
    """
    This function calculate region IOU when masks are
    inside different boxes
    Returns:
        intersection over unions of this two masks
    """
    #print(mask1.shape)
    #print(mask2.shape)
    
    
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x1 > x2 or y1 > y2:
        return 0
    w = x2 - x1 + 1
    h = y2 - y1 + 1
    # get masks in the intersection part
    start_ya = y1 - box1[1]
    start_xa = x1 - box1[0]
    inter_maska = mask1[start_ya: start_ya + h, start_xa:start_xa + w]

    start_yb = y1 - box2[1]
    start_xb = x1 - box2[0]
    inter_maskb = mask2[start_yb: start_yb + h, start_xb:start_xb + w]
    
    #plt.subplot(2,2,1) 
    #plt.imshow(inter_maska)
    #plt.subplot(2,2,2) 
    #plt.imshow(inter_maskb) 
    
    #print(inter_maska.shape)
    #print(inter_maskb.shape)
    
    assert inter_maska.shape == inter_maskb.shape

    inter = np.logical_and(inter_maskb, inter_maska).sum()
    union = mask1.sum() + mask2.sum() - inter
    if union < 1.0:
        return 0
    return float(inter) / float(union)

In [29]:
def bbox(part):
    mask = part.copy()    
    mask[mask>=1] = 1
    blob = np.where(mask>=1)
    x1 = min(blob[1])
    x2 = max(blob[1])
    y1 = min(blob[0])
    y2 = max(blob[0])
    
    box = [x1, y1, x2, y2]
    inst =  mask[y1:y2+1,x1:x2+1]
    
    return inst,box

In [30]:
def FetchScore(matches, part):
    maxoverlap = -1000
    targetscore = 0
    
    for oneinst in matches:
        if 'box' not in oneinst:
            score = oneinst[oneinst.rindex('_')+1:oneinst.index('.npy')]
            score = float(score)
            inst = np.load(oneinst)
            instbox = np.load(oneinst[:oneinst.index('.npy')]+'_box.npy')
            my_inst,my_instbox  = bbox(part)
            ov = mask_overlap_box(instbox,my_instbox,inst,my_inst)
            if ov>maxoverlap:
                targetscore = score
                maxoverlap = ov
                
    return targetscore

In [31]:
def GetAP(part,label,gt_part,gt_part_label,oneclass,imgname):

    test_label = classes.get(oneclass)    
    num_pred = 0
    for cls in label:
        if cls == classes.get(oneclass):
            num_pred = num_pred+1
            
    fp = np.zeros(num_pred)
    tp = np.zeros(num_pred)
    ov_areas = np.zeros(num_pred)    
    scores = np.zeros(num_pred)
    
    gt_cls_num = 0
    for cls in gt_part_label:
        if cls == classes.get(oneclass):
            gt_cls_num = gt_cls_num+1
            
    #print "instance count: "+ str(num_pred)
    gt_detect_list = np.zeros((len(gt_part), 1))
    
    all_ov = []
    
    idx = 0
    
    for i in range(0,len(part)):
        if label[i] != test_label:
            continue
        
        ###This instance is going to be in tp or fp, need score!!
        ###find top score candidates
        realname = imgname[:imgname.index('.')]
        queryfolder = fcis_path[:len(fcis_path)-1] +'_SCORE/' + classes.get(oneclass) +'/'
        queryname = realname +'_'+ '*'+ '.npy'  
        matches = list(search_file(queryname,queryfolder))
        scores[idx] = FetchScore(matches,part[i])
        if label[i] not in gt_part_label: #this label  is not in ground truth
            fp[idx] = 1
            continue
        cur_overlap = -1000
        cur_overlap_ind = -1
        for j in range(0,len(gt_part)):
            if gt_part_label[j] == label[i]:
                ov = mask_overlap(gt_part[j], part[i])
                if ov > cur_overlap:
                    cur_overlap = ov
                    cur_overlap_ind = j                    
        if cur_overlap == -1000:
            print('overlap==-1000')
            cur_overlap = 0
        ov_areas[idx]=cur_overlap
        if cur_overlap >= ov_thresh:
            if gt_detect_list[cur_overlap_ind]:
                fp[idx] = 1
            else:
                tp[idx] = 1
                gt_detect_list[cur_overlap_ind] = 1
        else:
                fp[idx] = 1
        idx = idx+1
    return tp,fp,ov_areas,scores,gt_cls_num

In [32]:
def GetAPwithOV(part,label,part_ov,part_ov_id,part_score, gt_part, gt_part_label,oneclass,imgname):
    test_label = classes.get(oneclass)    
    num_pred = 0
    for cls in label:
        if cls == classes.get(oneclass):
            num_pred = num_pred+1
            
    fp = np.zeros(num_pred)
    tp = np.zeros(num_pred)
    
    gt_cls_num = 0
    for cls in gt_part_label:
        if cls == classes.get(oneclass):
            gt_cls_num = gt_cls_num+1
            
    #print "instance count: "+ str(num_pred)
    gt_detect_list = np.zeros((len(gt_part), 1))
    
    idx = 0
    
    for i in range(0,len(part)):
        if label[i] != test_label:
            continue       
        score[idx] = part_score[i]
        
        if label[i] not in gt_part_label: #this label  is not in ground truth
            fp[idx] = 1
            continue
               
        cur_overlap = part_ov[i]
        cur_overlap_ind = part_ov_id[i]
        ov_areas[idx]=cur_overlap
        
        if cur_overlap >= ov_thresh:
            if gt_detect_list[cur_overlap_ind]:
                fp[idx] = 1
            else:
                tp[idx] = 1
                gt_detect_list[cur_overlap_ind] = 1
        else:
                fp[idx] = 1
        idx = idx+1

    return tp,fp,ov_areas,scores,gt_cls_num

In [33]:
def savefile(filename,  overlap):
    output = open(filename, 'w')
    #for n in ap:
    #   output.write(str(n)+' ')
    #output.write('\n')
    for n in overlap:
        output.write(str(n)+' ')
    output.close()

In [34]:
##########################################
#####the slowest part
#####separate each image into instances

In [35]:
#classes = {0: 'CubeBody',  1:'CubeFace', 2: 'CylinderBody',3:  'CylinderFace' ,4:'Handle'}
classes = {0: 'CubeBody',  1:'CubeFace', 2: 'CylinderBody',3:  'CylinderFace' }
num_classes = len(classes)
ov_thresh = 0.5
print(num_classes) 

4


In [59]:
#folderpath = 'ForEvaluationTest_CurrentGeoNetVal/ForEvaluationTest_CurrentGeoNetVal/'
folderpath = 'Exper_FCIS_DCN/'
fcis_path = folderpath + '11_15_RCNN_7_11_epoch_pkl_DCN/'

fcis_list = os.listdir(fcis_path)
print(len(fcis_list))

15399


In [60]:
txt_path = folderpath + '11_15_RCNN_7_11_epoch_pkl/'

In [54]:
idx= 0
instances_name = []
instances_cls = []
instances_score = []
fcis_part_all = []
fcis_part_label_all = []
fcis_part_score_all = []
fcis_withoutdcn_part_all = []

imageset_file = os.path.join(folderpath, 'val.txt')
with open(imageset_file, 'r') as f:
    lines = f.readlines()
image_names = [x.strip() for x in lines]

for imgname in image_names:
    print idx
    realname = imgname
    print realname
    txt = txt_path+realname+'.txt'
    print txt
    with open(txt, 'r') as f:
        lines = f.readlines()
    instances_name = [x[:x.index(' ')] for x in lines]
    instances_cls = [classes.get(int(x[x.index(' ')+1:x.rindex(' ')])-1) for x in lines]
    instances_score = [float(x[x.rindex(' '):]) for x in lines]
    
#     print instances_name
#     print instances_cls
#     print instances_score
    
    part = []    
    withoutdcn_part = []
    
    for name in instances_name:
        realname = name[:name.index('.png')] + '_seg.png'
        print fcis_path+realname
#         if not os.path.exists(fcis_path+realname):
#             continue        
        img = cv2.imread(fcis_path+name,1)
        img = img[:,:,0]
        img[img>0] = 1
#         plt.figure()
#         plt.imshow(img)
        part.append(img)
#         print folderpath + '11_12_RCNN_pkl/'+name
#         withoutdcn_img = cv2.imread(folderpath + '11_12_RCNN_pkl/'+name,1)
#         withoutdcn_img = withoutdcn_img[:,:,0]
#         withoutdcn_img[withoutdcn_img>0] = 1
#         withoutdcn_part.append(withoutdcn_img)
        
#         plt.figure()
#         plt.imshow(withoutdcn_img)
        
    idx = idx+1
#     print(str(idx) + " " + imgname) 
#     fcis_withoutdcn_part_all.append(withoutdcn_part)
    fcis_part_all.append(part)
    fcis_part_label_all.append(instances_cls)
    fcis_part_score_all.append(instances_score)

0
bal_00003
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00003.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00003_0_seg.png
1
bal_00007
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00007.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00007_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00007_1_seg.png
2
bal_00010
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00010.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00010_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00010_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00010_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00010_3_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00010_4_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00010_5_seg.png
3
bal_00011
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00011.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00011_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00011_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bal_00011_2_seg.png


52
bla_00029
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00029.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00029_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00029_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00029_2_seg.png
53
bla_00031
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00031.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00031_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00031_1_seg.png
54
bla_00033
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00033.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00033_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00033_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00033_2_seg.png
55
bla_00034
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00034.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00034_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00034_1_seg.png
56
bla_00044
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00044.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bla_00044

Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00024_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00024_2_seg.png
121
bow_00027
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027_3_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027_4_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027_5_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00027_6_seg.png
122
bow_00029
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00029.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00029_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00029_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00029_2_seg.png
123
bow_00031
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00031.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00031_0_seg.png


Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00143_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00143_2_seg.png
157
bow_00147
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00147.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00147_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00147_1_seg.png
158
bow_00152
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00152.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00152_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00152_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00152_2_seg.png
159
bow_00155
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00155.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00155_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00155_1_seg.png
160
bow_00161
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00161.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00161_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00161_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/bow_00161_2_

Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00137_1_seg.png
213
box_00145
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00145.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00145_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00145_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00145_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00145_3_seg.png
214
box_00155
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00155.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00155_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00155_1_seg.png
215
box_00160
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00160.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00160_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00160_1_seg.png
216
box_00164
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00164.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00164_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/box_00164_1_seg.png
217
box_00167
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pk

...

Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01930_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01930_3_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01930_4_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01930_5_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01930_6_seg.png
3329
cube_01931
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01931.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01931_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01931_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01931_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01931_3_seg.png
3330
cube_01933
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01933.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01933_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01933_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01933_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_01933_3_seg.png
3331
cube_01945
Exper_FCIS_DCN/11_15_RCNN_7_11_epo

Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02002_11_seg.png
3354
cube_02004
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02004.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02004_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02004_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02004_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02004_3_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02004_4_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02004_5_seg.png
3355
cube_02008
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02008.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02008_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02008_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02008_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02008_3_seg.png
3356
cube_02011
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02011.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02011_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pk

Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02125_1_seg.png
3386
cube_02127
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02127.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02127_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02127_1_seg.png
3387
cube_02128
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02128.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02128_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02128_1_seg.png
3388
cube_02131
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131_3_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131_4_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131_5_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02131_6_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl

Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02238_2_seg.png
3422
cube_02242
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02242.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02242_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02242_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02242_2_seg.png
3423
cube_02246
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02246.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02246_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02246_1_seg.png
3424
cube_02248
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02248.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02248_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02248_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02248_2_seg.png
3425
cube_02250
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02250.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02250_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02250_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11

Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02318_2_seg.png
3444
cube_02324
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_3_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_4_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_5_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_6_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02324_7_seg.png
3445
cube_02325
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02325.txt
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02325_0_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02325_1_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02325_2_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_02325_3_seg.png
Exper_FCIS_DCN/11_15_RCNN_7_11_epoch_pkl/cube_0232

In [55]:
print len(fcis_part_all)
print len(fcis_part_label_all)
print len(fcis_part_score_all)
print len(fcis_withoutdcn_part_all)

3474
3474
3474
0


### save pkl
1. det_file  按img_idx排序，每一行代表当前img的每一个detection，每一个detection为5维，前四维是xmin,ymin,xmax,ymax，最后一维是score
2. seg_file  按img_idx排序，每一行代表当前img的每一个detection对应的segmentation, 在bbox内的mask, 有每一个pixel的score, 改成0/1


In [56]:
def GetPKLline(part,part_label,part_score,imgname,cls_name):    
    det_line = []
    seg_line = []
        
    for i in range(0,len(part)):
        oneclass = part_label[i]
        if oneclass == cls_name:
            score = part_score[i]     
            
            if part[i].sum()==0:
                continue
                        
            inst,box = bbox(part[i])
            
#             plt.figure()
#             plt.imshow(inst)
            
            temp = np.zeros(5)
            temp[0:4] = box
            temp[-1] = score
            
            inst = np.array(inst)
            
            det_line.append(temp)
            seg_line.append(inst)
        
    return det_line,seg_line

In [58]:
fcis_part_ov_all = []
fcis_part_ov_id_all = []
for j in range(0,len(classes)):
    det_file = []
    seg_file = []
    print(classes.get(j))
    for i in range(0,len(fcis_part_all)):
        print i
        part = fcis_part_all[i]
#         withoutdcn_part = fcis_withoutdcn_part_all[i]
        part_label = fcis_part_label_all[i]
        part_score = fcis_part_score_all[i]
        det_line,seg_line= GetPKLline(part,part_label,part_score,image_names[i],classes.get(j))
    
#         print(det_line)
        
        #if len(det_line)>0 and len(seg_line)>0:
        det_file.append(det_line)
        seg_file.append(seg_line)
    
    output_det = open(fcis_path+'new/'+ classes.get(j)+'_det.pkl','wb')
    pickle.dump(det_file,output_det)
    output_det.close()
    
    output_seg = open(fcis_path+'new/'+ classes.get(j)+'_seg.pkl','wb')
    pickle.dump(seg_file,output_seg)
    output_seg.close()

CubeBody
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
2

1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142


181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430


2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587
2588


732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981


2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587
2588
2589
2590
2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681


923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
11

2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791


In [66]:
print(det_file)

with open(fcis_path+'CylinderFace_seg.pkl', 'rb') as f:
    masks_pkl = pickle.load(f)

[[], [], [array([  57.        ,  270.        ,   85.        ,  285.        ,
          0.85410053])], [], [array([ 260.        ,   75.        ,  381.        ,   93.        ,
          0.86683512])], [], [array([ 211.        ,  227.        ,  218.        ,  228.        ,
          0.94645309])], [], [], [array([ 178.        ,  213.        ,  271.        ,  254.        ,
          0.91816694])]]


In [54]:
print(len(fcis_part_all))
print(len(fcis_part_score_all))
print(len(fcis_part_ov_all))
print(len(fcis_part_ov_id_all))

3474
11747
11747
11747


In [56]:
#ov_thresh =   0.5 #overlap_threshold - 0.5/0.7....
ov_thresh = np.arange(0.0,1.0,0.05)

In [ ]:
ap_all = []
map_all = []

for one_ov in ov_thresh[10:11]:
    all_tp = [[],[],[],[],[]]
    all_fp = [[],[],[],[],[]]
    all_scores = [[],[],[],[],[]]
    gt_cls_num = np.zeros(num_classes)
    
    for oneclass in range(0,num_classes):
        print(classes.get(oneclass))
        #find all instances in this class
        for i in range(0,len(fcis_part_all)):
            if i %1000 == 0:
                print(i)
                
            part = fcis_part_all[i]
            part_label = fcis_part_label_all[i]
            part_ov = fcis_part_ov_all[i]
            part_ov_id = fcis_part_ov_id_all[i]
            part_score = fcis_part_score_all[i]

            gt_part = gt_part_all[i]
            gt_part_label = gt_part_label_all[i]

            this_cls =  classes.get(oneclass)
            if this_cls in part_label:
                tp, fp , ov_areas , scores ,gt_thiscls_num= GetAPwithOV(part,part_label,part_ov,part_ov_id,part_score,gt_part,gt_part_label,oneclass,gt_list[i])
                #print(scores)
                #print(ov_areas)
                all_tp[oneclass].extend(tp)
                all_fp[oneclass].extend(fp)
                overlap_areas[oneclass].extend(ov_areas)
                all_scores[oneclass].extend(scores)
                gt_cls_num[oneclass] = gt_cls_num[oneclass] + gt_thiscls_num              
        
    ap = np.zeros(num_classes)
    for oneclass in range(0,num_classes):
        ####### 按照score重新对tp 和 fp 排序
        orderidx = np.argsort(all_scores[oneclass])
        orderidx = orderidx[::-1]
        tps_backup = all_tp[oneclass][:]
        fpt_backup = all_fp[oneclass][:]
        or_tps = []
        or_fps = []
        for x in orderidx:
            or_tps.append(all_tp[oneclass][x])
            or_fps.append(all_fp[oneclass][x])
            one_tp = or_tps[:]
            one_fp = or_fps[:]
        ##################################

        #one_tp = all_tp[oneclass]
        #one_fp = all_fp[oneclass]

        num_pos = gt_cls_num[oneclass]
        one_tp = np.cumsum(one_tp)
        one_fp = np.cumsum(one_fp)
        rec = one_tp / np.maximum(float(num_pos),  np.finfo(np.float64).eps)
        #avoid divide by zero in case the first matches a difficult gt
        prec = one_tp / np.maximum(one_fp+one_tp, np.finfo(np.float64).eps)
        ap[oneclass] = voc_ap(rec, prec, True)
        
    print('FCIS')
    print('~~~~~~~~~~~~~~~~~~~~~~~')
    mAP = 0.0
    for oneclass in range(0,num_classes):
        mAP+=ap[oneclass]
        print('AP for ' + classes.get(oneclass) +" =  " + str(ap[oneclass]*100))

    print('~~~~~~~~~~~~~~~~~~~~~~~')
    mAP /= num_classes
    print('Mean AP@' + str(ov_thresh) +' = ' + str(mAP*100))

    map_all.append(mAP)
    ap_all.append(ap)
                
                

In [ ]:
#######################################################FCIS
all_tp = [[],[],[],[],[]]
all_fp = [[],[],[],[],[]]
overlap_areas = [[],[],[],[],[]]
all_scores = [[],[],[],[],[]]
gt_cls_num = np.zeros(num_classes)

for oneclass in range(0,num_classes):
    print(classes.get(oneclass))
    #find all instances in this class
    for i in range(0,len(fcis_part_all)):
        if i %1000 == 0:
            print(i)
        part = fcis_part_all[i]
        part_label = fcis_part_label_all[i]
        
        gt_part = gt_part_all[i]
        gt_part_label = gt_part_label_all[i]
        
        this_cls =  classes.get(oneclass)
        if this_cls in part_label:
            tp, fp , ov_areas , scores ,gt_thiscls_num= GetAP(part,part_label,gt_part,gt_part_label,oneclass,gt_list[i])
            #print(scores)
            #print(ov_areas)
            all_tp[oneclass].extend(tp)
            all_fp[oneclass].extend(fp)
            overlap_areas[oneclass].extend(ov_areas)
            all_scores[oneclass].extend(scores)
            gt_cls_num[oneclass] = gt_cls_num[oneclass] + gt_thiscls_num

CubeBody
0
1000


In [ ]:
ap = np.zeros(num_classes)
for oneclass in range(0,num_classes):
    
    ####### 按照score重新对tp 和 fp 排序
    orderidx = np.argsort(all_scores[oneclass])
    orderidx = orderidx[::-1]
    tps_backup = all_tp[oneclass][:]
    fpt_backup = all_fp[oneclass][:]
    or_tps = []
    or_fps = []
    for x in orderidx:
        or_tps.append(all_tp[oneclass][x])
        or_fps.append(all_fp[oneclass][x])
        one_tp = or_tps[:]
        one_fp = or_fps[:]
    ##################################

    #one_tp = all_tp[oneclass]
    #one_fp = all_fp[oneclass]
    
    num_pos = gt_cls_num[oneclass]
    one_tp = np.cumsum(one_tp)
    one_fp = np.cumsum(one_fp)
    rec = one_tp / np.maximum(float(num_pos),  np.finfo(np.float64).eps)
    #avoid divide by zero in case the first matches a difficult gt
    prec = one_tp / np.maximum(one_fp+one_tp, np.finfo(np.float64).eps)
    ap[oneclass] = voc_ap(rec, prec, True)
print('FCIS')
print('~~~~~~~~~~~~~~~~~~~~~~~')
mAP = 0.0
for oneclass in range(0,num_classes):
    mAP+=ap[oneclass]
    print('AP for ' + classes.get(oneclass) +" =  " + str(ap[oneclass]*100))
    
print('~~~~~~~~~~~~~~~~~~~~~~~')
mAP /= num_classes
print('Mean AP@' + str(ov_thresh) +' = ' + str(mAP*100))

In [46]:
fcis_dcn_noss_score_backup = all_scores[:]
fcis_dcn_noss_part = fcis_part_all[:]
fcis_dcn_noss_part_label = fcis_part_label_all[:]
fcis_dcn_noss_tp_5 = all_tp[:]
fcis_dcn_noss_fp_5 = all_fp[:]

In [70]:
############################################################################
####save each instance as images

In [77]:
def save_inst(fcis_part_all,fcis_part_label_all,folder):
    os.mkdir(folderpath+folder)
    for i in range(0,len(fcis_part_all)):
        imgname = img_names[i]
        oneimg = fcis_part_all[i]
        oneimg_label = fcis_part_label_all[i]
        for j in range(0,len(oneimg)):
            inst = oneimg[j]
            inst[inst>=1] = 255
            inst_label = oneimg_label[j]
            name = imgname[:imgname.index('.')]
            filename = folderpath +folder+'/'+ name + '_'+str(j)+'_' + inst_label +'.png'
            print(filename)
            cv2.imwrite(filename, inst)

In [78]:
img_names = fcis_list
#save_inst(gt_part_all,gt_part_label_all,'gt_split')
#save_inst(fcis_part_all,fcis_part_label_all,'fcis_split')
save_inst(geonet_part_all,geonet_part_label_all,'fcis_dcn_noss_split')

FileExistsError: [Errno 17] File exists: 'Exper_FCIS_DCN/fcis_dcn_noss_split'

In [ ]:
#########################################################

In [ ]:
##############################################################################
####save per image overlap

In [23]:
def PerImageOV(part,label,gt_part,gt_label):
    # part is one image
    ov = np.zeros(len(part))
    gt_detect_list = np.zeros(len(gt_part))
    for i in range(0,len(part)):
        cur_overlap = -1000
        curr_overlap_idx = -1
        for j in range(0,len(gt_part)):
            if gt_label[j] == label[i]:
                ovv = mask_overlap(gt_part[j], part[i])
                if ovv > cur_overlap:
                    cur_overlap = ovv
                    curr_overlap_idx = j
        if cur_overlap == -1000:
            ov[i] = 0
        else:
            if gt_detect_list[curr_overlap_idx]==0:
                ov[i] = cur_overlap
                gt_detect_list[curr_overlap_idx]==1
            else:
                ov[i] = 0
    return ov

In [24]:
counter = 0
fcis_mean_ov = 0
for i in range(0,len(fcis_part_all)):
    #print(i)
    ov = PerImageOV(fcis_part_all[i],fcis_part_label_all[i],gt_part_all[i],gt_part_label_all[i])
    print(ov)
    for x in ov:
        counter = counter + 1
        fcis_mean_ov = x + fcis_mean_ov    
    #filename = fcis_list[i]
    #savefile(folderpath+'fcis/'+filename[:filename.index('.')]+'.txt',ov)

[ 0.65633475  0.        ]
[ 0.2268159]
[ 0.         0.         0.8578362]
[ 0.          0.83039648]
[ 0.854035    0.93994082  0.38944717]
[ 0.  0.  0.  0.  0.  0.]
[]
[ 0.]
[ 0.  0.]
[ 0.88254629]
[ 0.  0.  0.]
[ 0.]
[ 0.92394334  0.94042572]
[ 0.847061    0.42210667  0.49913993]
[ 0.93086817  0.9244508 ]
[ 0.88640174  0.9313855 ]
[ 0.78910798  0.89674101]
[ 0.94015342  0.89470025]
[ 0.76597422  0.71728992]
[ 0.82746927  0.88383205]
[ 0.]
[ 0.8146519   0.77155701]
[ 0.9117041   0.88244986]
[ 0.  0.  0.  0.]
[ 0.85152517  0.93329168]
[ 0.89063555  0.91015096  0.64011856  0.62854331]
[ 0.         0.         0.         0.6589698]
[ 0.  0.  0.]
[ 0.33955438  0.        ]
[ 0.92060165  0.94998108]
[ 0.87027353]
[ 0.61251139]
[ 0.94455703  0.93239158]
[ 0.76355113  0.        ]
[ 0.  0.]
[ 0.9565762   0.86868742]
[ 0.59809524  0.          0.79316338]
[ 0.  0.  0.]
[ 0.          0.          0.72402419]
[ 0.01028504  0.5402867   0.34503511]
[ 0.         0.1224391]
[ 0.  0.]
[ 0.          0.     

[ 0.10155908  0.57458162  0.66603641]
[ 0.]
[ 0.92564821  0.91039042]
[ 0.  0.]
[ 0.         0.         0.7237112]
[ 0.67974649]
[ 0.02078378]
[ 0.87783661  0.94946145]
[ 0.69981527  0.89473929]
[ 0.94463351  0.87895546]
[ 0.82978598  0.94060735]
[ 0.19965572]
[ 0.]
[ 0.         0.         0.8222406]
[ 0.75887691  0.90887393]
[ 0.          0.          0.          0.87526055]
[ 0.          0.79269016]
[ 0.          0.          0.84184823  0.          0.        ]
[ 0.87649078  0.82990084]
[ 0.94104301  0.00698617]
[ 0.89759937  0.68680472]
[ 0.  0.  0.]
[ 0.92302746  0.93367706]
[ 0.86221837  0.91172125]
[ 0.          0.          0.91698924]
[ 0.          0.          0.73590316  0.85227816]
[ 0.          0.          0.          0.69634862]
[ 0.  0.  0.]
[ 0.60495522  0.74917492  0.88497557  0.77192685  0.86723677]
[ 0.95093624  0.91757878]
[ 0.  0.  0.  0.]
[ 0.91440956  0.84293955]
[ 0.          0.89540059]
[ 0.8727133   0.72661576  0.05651309]
[ 0.          0.87123288]
[ 0.73638968  0.

[ 0.51156495  0.45092025  0.74352367]
[ 0.94719017  0.9388322 ]
[ 0.94292913  0.93241806]
[ 0.95178566  0.92226633]
[  8.74655225e-01   2.83173812e-05   8.89420035e-01]
[ 0.91938216  0.8758209 ]
[ 0.95246754  0.94283554]
[ 0.         0.         0.         0.4375975]
[ 0.  0.]
[ 0.  0.]
[ 0.94855352  0.95153506]
[]
[ 0.          0.          0.64415387  0.4673256 ]
[ 0.76178997  0.95211688]
[ 0.9506228   0.95422128]
[ 0.]
[]
[ 0.  0.]
[ 0.88014383  0.9265506 ]
[ 0.92721843  0.56935687]
[ 0.]
[ 0.]
[]
[ 0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.]
[ 0.  0.  0.]
[ 0.19944598  0.709458    0.75311651]
[ 0.90623726  0.96015969]
[ 0.]
[ 0.  0.]
[ 0.8462847   0.85456148  0.87501147  0.77202177]
[ 0.]
[ 0.         0.1135165  0.700625 ]
[ 0.714791    0.94154403]
[]
[ 0.56601307  0.921262  ]
[ 0.93829504  0.94664752]
[ 0.]
[ 0.88807683  0.88503399]
[ 0.89662677  0.90153779]
[ 0.94017592  0.91855726]
[ 0.  0.]
[ 0.          0.          0.80878534  0.70619776]
[ 0.50152905  0.96989308]
[ 0.  0.]
[ 0.      

[ 0.7474414  0.       ]
[ 0.  0.  0.]
[ 0.77809147  0.8394209 ]
[ 0.76689446  0.95078969]
[ 0.91211443  0.90409743]
[ 0.          0.          0.85770841]
[ 0.80995666  0.86959031]
[ 0.  0.]
[ 0.91961399  0.8989138 ]
[ 0.          0.          0.          0.92876929]
[ 0.          0.          0.90813911  0.9025695 ]
[ 0.  0.]
[ 0.43338584]
[ 0.91839992  0.89751027]
[ 0.92424242  0.94157759]
[ 0.68635494  0.83307573]
[ 0.]
[ 0.89973115  0.88770842]
[ 0.          0.          0.          0.          0.83943089  0.84099617]
[ 0.  0.  0.]
[ 0.09250109  0.07908376  0.        ]
[ 0.  0.  0.  0.]
[ 0.57659128  0.78664424]
[ 0.  0.]
[ 0.          0.          0.          0.76722133]
[ 0.95473428  0.9225923 ]
[ 0.86676893]
[ 0.          0.          0.79326701]
[ 0.93009915  0.8990954 ]
[ 0.76206724  0.84192298]
[ 0.77891216  0.71519443]
[ 0.]
[ 0.88989057  0.83071931]
[ 0.]
[ 0.8928278   0.02419001]
[ 0.90811674  0.9238002   0.73355956]
[ 0.94424278  0.95209903]
[ 0.]
[]
[ 0.91991716]
[ 0.91311086 

[ 0.         0.7245509  0.       ]
[]
[ 0.]
[ 0.          0.91034913]
[ 0.57729576  0.70288079]
[ 0.92450204  0.94273054]
[ 0.  0.]
[ 0.01378868  0.        ]
[ 0.  0.]
[ 0.87124464  0.91332888]
[ 0.71668956  0.          0.78964942]
[ 0.  0.]
[ 0.          0.          0.73170732]
[ 0.  0.]
[ 0.  0.  0.]
[]
[ 0.          0.          0.55234657]
[ 0.  0.  0.  0.]
[ 0.  0.  0.]
[ 0.90592972  0.63560034]
[ 0.91515391  0.92275208]
[ 0.  0.]
[ 0.73680782  0.92880863]
[ 0.54466403  0.90832063]
[ 0.          0.          0.82321188  0.82360529  0.          0.        ]
[ 0.93271462  0.95200259]
[ 0.19850352]
[ 0.]
[ 0.          0.83003184]
[ 0.00639397  0.09404117]
[]
[ 0.91136803  0.72253993]
[ 0.  0.  0.  0.]
[ 0.0068262   0.16074864]
[ 0.  0.  0.  0.  0.]
[ 0.84481715  0.94511246]
[ 0.          0.          0.82979452]
[ 0.          0.          0.4461057   0.38398177]
[ 0.91200342  0.90438717]
[ 0.92275997  0.93283344]
[]
[ 0.9265796   0.91712573]
[ 0.]
[ 0.          0.          0.80132309]
[ 0

...

[ 0.92892114  0.94526564]
[ 0.  0.]
[ 0.          0.          0.          0.90333158]
[ 0.57931625  0.80592654  0.97060406]
[ 0.94293118  0.95381685]
[ 0.]
[ 0.         0.         0.         0.         0.         0.7486965]
[ 0.  0.  0.]
[ 0.          0.          0.66610169  0.65840708]
[ 0.90538807  0.92077857]
[ 0.          0.          0.56570672]
[ 0.  0.]
[ 0.82891285  0.93702165]
[ 0.  0.  0.]
[ 0.          0.8081903   0.77986023  0.          0.        ]
[]
[ 0.  0.]
[ 0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.76320378  0.63926262]
[ 0.56868795  0.91996927]
[ 0.  0.  0.]
[ 0.          0.92252102]
[ 0.  0.]
[ 0.          0.          0.62249428]
[ 0.90663043  0.87982011]
[ 0.66263007  0.78843534  0.8770201 ]
[ 0.          0.          0.69247432]
[ 0.          0.          0.          0.80246092]
[ 0.45883642]
[ 0.88525495  0.95039041]
[ 0.          0.          0.72433853]
[ 0.73413109  0.56024707]
[]
[ 0.93977901  0.95511487]
[ 0.90575472  0.7538137 ]
[ 0.  0.]
[]
[ 0.01156677  0.51925667]

[ 0.  0.  0.  0.  0.]
[ 0.001853    0.59153036]
[ 0.  0.  0.  0.]
[ 0.          0.          0.84961619]
[ 0.]
[ 0.  0.  0.]
[ 0.          0.76087336]
[ 0.78823529  0.88036847]
[ 0.90505695  0.88146796]
[ 0.83289538  0.9063426   0.65059113  0.78599171]
[ 0.          0.40802213  0.          0.48316062  0.83335949]
[ 0.90273849  0.88769656]
[ 0.88194052  0.        ]
[ 0.72163644  0.          0.        ]
[ 0.95771151  0.94213218]
[ 0.93356101  0.9375    ]
[ 0.8622975   0.85714286  0.83320979  0.90667945  0.86587302  0.88881748
  0.82972136  0.85577841  0.90193506  0.8896481 ]
[ 0.61584924]
[ 0.  0.]
[ 0.74279413  0.45008723]
[ 0.23742561  0.88627668]
[ 0.  0.  0.  0.]
[ 0.04948467]
[ 0.  0.]
[ 0.          0.          0.91693009]
[ 0.  0.]
[ 0.          0.62867012  0.59791123  0.          0.8608522 ]
[ 0.          0.          0.7538674   0.          0.75672737  0.
  0.55834464  0.77358491]
[ 0.          0.          0.78108652]
[ 0.91871567  0.70625541]
[ 0.  0.]
[ 0.  0.]
[ 0.51392703]
[ 0.

[ 0.94448617  0.94906612]
[ 0.87793255  0.90730306]
[ 0.]
[ 0.          0.          0.79432283]
[ 0.92432044  0.92424455]
[ 0.85330619]
[ 0.          0.          0.91778793]
[ 0.         0.6955267]
[ 0.95322456  0.90363563]
[ 0.85384869  0.91231793]
[ 0.80339335  0.52654806]
[ 0.91440589  0.94617429]
[ 0.85186979  0.90213261]
[ 0.83157895  0.92537961  0.81152153  0.7719797 ]
[ 0.          0.          0.79468098  0.60830527]
[ 0.2127712]
[ 0.36569482  0.50312278]
[ 0.          0.          0.76720383]
[ 0.01240038]
[ 0.  0.  0.]
[ 0.]
[ 0.96276052  0.96423643]
[ 0.          0.78810409]
[ 0.79832854  0.          0.81087342  0.92901054]
[ 0.  0.  0.  0.  0.  0.]
[ 0.85884597  0.93071805]
[ 0.  0.]
[ 0.82351852  0.54209919  0.87793081  0.71799333  0.92797235]
[ 0.          0.          0.69395263  0.71546547]
[ 0.92929964  0.94437931]
[ 0.90598925  0.9167005 ]
[ 0.          0.          0.          0.65132188]
[ 0.82921404  0.          0.        ]
[ 0.95041737  0.94935971]
[ 0.          0.   

[ 0.78132716  0.8264922   0.69468522  0.83168883  0.8421362   0.74149719]
[ 0.  0.  0.]
[ 0.715       0.43196106]
[ 0.  0.  0.]
[ 0.87429719  0.88167748]
[ 0.7856305   0.80138356]
[ 0.89557425]
[ 0.  0.  0.]
[ 0.9042525   0.79996946]
[ 0.64596162]
[ 0.          0.78891616  0.96494089]
[ 0.]
[ 0.          0.          0.49576562]
[ 0.          0.          0.86073574]
[]
[ 0.          0.57592154  0.64067914  0.          0.93242187]
[ 0.          0.70921986]
[ 0.          0.          0.          0.84041184]
[ 0.          0.          0.          0.91204589]
[ 0.          0.          0.91569628]
[ 0.          0.          0.38320995]
[ 0.  0.]
[ 0.          0.58657613]
[ 0.94001751  0.91963048]
[ 0.89592998  0.82428168]
[ 0.]
[ 0.          0.          0.83615085]
[ 0.  0.]
[ 0.92327044  0.81393868]
[ 0.  0.  0.  0.]
[ 0.  0.]
[ 0.          0.80327528  0.        ]
[ 0.22895395]
[ 0.          0.          0.71697866]
[ 0.          0.          0.          0.80041524  0.84874059]
[ 0.00230909  0. 

[ 0.95116755  0.91624638]
[ 0.93237864  0.92017536]
[ 0.94816812  0.93580229]
[ 0.          0.          0.58735699  0.76277017]
[ 0.76769626]
[]
[ 0.09248848]
[ 0.  0.  0.  0.  0.]
[ 0.          0.          0.53007519  0.82315064]
[ 0.90161991  0.95957011]
[ 0.78699734  0.90353644]
[ 0.]
[ 0.86582084  0.88249922]
[ 0.  0.  0.]
[ 0.          0.          0.70423163  0.68654482]
[ 0.00094829  0.86540711]
[ 0.93917949  0.90514517]
[ 0.93172578  0.93735172]
[ 0.49044869  0.        ]
[ 0.92724523  0.88268233]
[ 0.          0.          0.84434762]
[ 0.  0.]
[ 0.6471364]
[]
[ 0.4572532   0.42671255  0.81421791]
[ 0.91667617  0.95320901]
[ 0.  0.]
[ 0.80182232  0.96418323]
[ 0.]
[ 0.  0.]
[ 0.57180414  0.92933492]
[ 0.          0.          0.42039106]
[ 0.  0.]
[ 0.85567672  0.82210243]
[ 0.          0.          0.88313062]
[ 0.          0.          0.71010542]
[ 0.95969905  0.96537458]
[ 0.37102459  0.        ]
[ 0.90906314  0.92807902]
[ 0.91330588]
[ 0.          0.          0.73295198]
[ 0.7

In [25]:
print(fcis_mean_ov)
print(counter)
print(fcis_mean_ov/(2*counter))

3514.61452873
8355
0.210330013688


In [32]:
counter = 0
geonet_mean_ov = 0
imgovlist = []
for i in range(0,len(geonet_part_all)):
    #print(i)
    ov = PerImageOV(geonet_part_all[i],geonet_part_label_all[i],gt_part_all[i],gt_part_label_all[i])
    if 'CylinderBody' in geonet_part_label_all[i]:
        imgovlist.append(ov.mean())
        print(ov)
    else:
        imgovlist.append(0)
    for x in ov:
        counter = counter + 1
        geonet_mean_ov = x + geonet_mean_ov
        
    #filename = geonet_list[i]
    #savefile(folderpath+'geonet/'+filename[:filename.index('.')]+'.txt',ov)

[ 0.50966481  0.79569456  0.83667883  0.84591008]
[ 0.69959862  0.93715898  0.80275892]
[ 0.96122158  0.91028876]
[ 0.57677903  0.33146067  0.8300123   0.89010283  0.87571893  0.52678571
  0.74451411]
[ 0.60400407  0.90558369  0.71456486]
[ 0.03359986  0.          0.05113863  0.13390207]
[ 0.62285273  0.7975776   0.        ]
[ 0.80239521  0.93168626  0.81852753  0.51553484]
[ 0.92695288  0.74889734  0.82586922  0.8783952 ]
[ 0.63139932  0.37595908  0.79815016  0.42119565  0.86476523  0.
  0.04471545]
[ 0.0357922   0.8302583   0.88393852  0.57092496  0.00342857]
[ 0.93849613  0.60744451  0.66900621  0.72157153  0.        ]
[ 0.16779067  0.          0.89081747  0.87615601]
[ 0.67360645  0.81848875  0.01775457  0.97279811  0.86763479  0.74743076]
[ 0.57436276  0.69156515]
[ 0.63009404  0.76841778  0.70883755  0.53562082  0.88762737  0.87751371]
[ 0.          0.87550882  0.          0.72321429]
[ 0.79727736  0.81809485  0.68692372  0.74046961]
[ 0.9515636   0.84839238  0.11667439]
[ 0.    

[ 0.87769357  0.89169288  0.81668336  0.73824724]
[ 0.80603843  0.81946732  0.46006262]
[ 0.57971014  0.88660742  0.68068021  0.7635028   0.7585335   0.32669592
  0.84326923  0.62591067  0.24290221]
[ 0.71410929  0.90713811  0.86521377]
[ 0.75248381  0.85518835  0.50138497]
[ 0.00642769  0.8716233   0.78358789  0.85058035  0.54291819  0.61284567
  0.46615769  0.65492745  0.63764479]
[ 0.72988104  0.84107286  0.49929677  0.89018975]
[ 0.68156109  0.          0.74746699  0.79755829  0.63755013  0.7834385
  0.6931661 ]
[ 0.73577982  0.89563541  0.        ]
[ 0.65051903  0.94220183  0.93798988  0.81488473  0.73797282  0.73411838
  0.44825376]
[ 0.88180113  0.66666667  0.76515152  0.60243408  0.87180723]
[ 0.31029847  0.          0.72682927]
[ 0.89122807  0.9718946   0.83681399  0.82166247  0.78606692]
[ 0.69460803  0.83886389  0.80959582  0.83296214]
[ 0.70653256  0.65480696  0.10794894  0.80545571  0.80456656]
[ 0.89121339  0.88748287  0.2620539 ]
[ 0.51068534  0.          0.90745624  0.9

[ 0.78969774  0.90360199  0.          0.82150376]
[ 0.75547425  0.58508957  0.01583235  0.69690011  0.71990307]
[ 0.63994222  0.91251993  0.83451129]
[ 0.62175765  0.04597701  0.39918534  0.90033317  0.89878163]
[ 0.80126423  0.86005421  0.60651587]
[ 0.5152439   0.89439853  0.75055782]
[ 0.60935857  0.70676692  0.93167631  0.91634063  0.82238866  0.9268619 ]
[ 0.59079284  0.          0.77684324  0.84450294  0.6978332   0.77534006]
[ 0.82135186  0.78490893  0.86775526]
[ 0.5519495   0.          0.61873007  0.61392072  0.94316715  0.6901726 ]
[ 0.03085794  0.51683081  0.83638346  0.75297144]
[ 0.81600832  0.91280366  0.89191489  0.87610472  0.9313622   0.70192726]
[ 0.90465296  0.69954835]
[ 0.          0.47738693  0.78806584  0.        ]
[ 0.          0.          0.7381717   0.89661882  0.92699758]
[ 0.72703973  0.51724727  0.57017544]
[ 0.79904824  0.90593552  0.73192436  0.75804575  0.81794498]
[ 0.93310507  0.9380989   0.          0.60180523]
[ 0.87497249  0.          0.70390782]
[ 

[ 0.83777365  0.81644709  0.69535881]
[ 0.]
[ 0.55264578  0.          0.86056854]
[ 0.83344972  0.93878772  0.512303    0.7116816   0.77859949]
[ 0.92921493  0.92777971  0.81030702  0.91074596  0.7107608   0.72989172]
[ 0.93979163  0.80860801  0.        ]
[ 0.58386838  0.86470897  0.75058324  0.30117108]
[ 0.79159105  0.79966997  0.8959276   0.        ]
[ 0.88256387  0.96540122  0.86237374]
[ 0.91623667  0.60920831  0.90119698  0.86705989]
[ 0.88625904  0.7664042   0.6112577   0.90973843  0.8600242 ]
[ 0.8078853   0.8369338   0.60151007  0.83212268]
[ 0.88637261  0.77944496  0.7793563   0.57516834]
[ 0.94520366  0.74963181  0.82554862  0.88781897]
[ 0.84740154  0.70562454]
[ 0.53899309  0.84147575  0.          0.47907099]
[ 0.57698962  0.88373847  0.79738872  0.57755923  0.82088096  0.48564103
  0.12601532  0.41225242]
[ 0.86878453  0.88753094  0.85401174  0.89878105  0.7825905   0.82716049
  0.77087287]
[ 0.74328842  0.        ]
[ 0.94358732  0.90878378  0.79458863  0.71413102  0.8399

[ 0.82827039  0.81528055  0.73874126  0.67626722  0.74137003  0.65683539]
[ 0.          0.82245253  0.          0.          0.80145981]
[ 0.71580817  0.89259824  0.48797737  0.65174825]
[ 0.52601432  0.61659389  0.57896472  0.76961353  0.56934026]
[ 0.00323104  0.62698413  0.88263572  0.58204334  0.          0.        ]
[ 0.92949757  0.85438972]
[ 0.92907496  0.85196248  0.71106434]
[ 0.95045391  0.8784141   0.81783587  0.60475884]
[ 0.00438711  0.76137113]
[ 0.73113381  0.89128976  0.81132563]
[ 0.89994272  0.79456849]
[ 0.32271681  0.79649583  0.93587084  0.7311344   0.          0.          0.        ]
[ 0.62337662  0.90651801  0.89510007  0.78777819  0.52545249]
[ 0.90467472  0.96932409  0.84180871  0.91198623]
[ 0.38595401  0.92889634  0.36793848]
[ 0.68947018  0.88288509]
[ 0.          0.40328791  0.90001462  0.89873282  0.93548247  0.91466062
  0.66730465]
[ 0.94658449  0.63729746  0.84630292  0.28326679]
[ 0.6751269   0.59544749  0.64692321  0.90975884  0.89762115]
[ 0.69728601 

...

[ 0.80868842  0.81016206  0.83323039  0.68100713  0.59141247]
[ 0.96888498  0.78752086  0.8707269   0.87983418]
[ 0.95211609  0.09101494  0.90185827  0.82625164  0.          0.81173491
  0.83903302  0.48716343]
[ 0.73827328  0.77278426  0.88610285  0.94288496  0.77064362  0.82918319]
[ 0.          0.93148447  0.47511291  0.        ]
[ 0.75458538  0.8171123   0.31519346  0.60827251]
[ 0.625       0.95161807  0.79904306]
[ 0.62920489  0.88308113  0.84717198  0.76159779  0.39358188  0.78721238]
[ 0.88073394  0.80400798  0.53133469  0.86382979]
[ 0.55528667  0.87576119  0.79384944  0.7159581 ]
[ 0.91922631  0.86121324  0.73732719  0.67488912  0.        ]
[ 0.34867197  0.70342707  0.73636225  0.85437816  0.73368921  0.88548057]
[ 0.          0.          0.          0.85358005]
[ 0.59785638  0.82256077  0.9151463   0.31785817]
[ 0.83822339  0.92779414  0.81575173]
[ 0.62541413  0.92898226  0.34101767]
[ 0.67825575  0.95418363  0.8360302 ]
[ 0.          0.83793911  0.76611781  0.94329518  0.7

[ 0.61543799  0.67480243  0.          0.        ]
[ 0.92817995  0.89002157  0.74842371]
[ 0.58088235  0.95653516  0.75509017]
[ 0.86764935  0.52526122  0.89752504]
[ 0.          0.91193331  0.58735689  0.9463586 ]
[ 0.71980609  0.87353151  0.02809334  0.00980249  0.39947061]
[ 0.85386177  0.63236299  0.82290554  0.83915321  0.334305    0.51782499]
[ 0.87021909]
[ 0.61967695  0.70176335  0.6446381   0.87367297  0.85480349]
[ 0.75342466  0.87109712  0.        ]
[ 0.74744514  0.        ]
[ 0.80885569  0.93013348  0.63596059]
[ 0.584       0.92670342  0.          0.90210843]
[ 0.          0.83173202  0.          0.81787407  0.80608365  0.89731463
  0.89884148  0.80879121]
[ 0.93112098  0.78826165  0.93940511  0.83354997  0.83542191  0.89054373]
[ 0.9098863   0.          0.82887267]
[ 0.94314104  0.69386348  0.46372068]
[ 0.67411062  0.87646207  0.81137993]
[ 0.25316456  0.78157163  0.89105582  0.80911396  0.86671418  0.        ]
[ 0.7700813   0.90791234  0.86694476]
[ 0.5503265   0.6268257

[ 0.50959898  0.3724981   0.86019494  0.73174891  0.          0.11538462]
[ 0.84302789  0.89462844  0.          0.74536432]
[ 0.91118548  0.62570073  0.85410085]
[ 0.          0.84310279  0.85698866  0.60436351]
[ 0.84799107  0.86773184  0.89755223  0.90951572]
[ 0.48133878  0.          0.        ]
[ 0.75544794  0.88708838  0.        ]
[ 0.          0.86748481  0.96774308  0.86104061]
[ 0.14103261  0.9013274   0.93893484  0.88403382  0.8609358   0.73706708]
[ 0.00529714  0.          0.74618488  0.          0.          0.        ]
[ 0.86736313  0.8898857   0.77086304]
[ 0.84747374  0.82638196  0.87285201  0.51340338  0.          0.        ]
[ 0.81290909  0.85932246  0.07039297  0.77593864  0.72877774  0.88805031]
[ 0.89133517  0.90465628  0.          0.        ]
[ 0.48844521  0.82395269  0.63612663  0.33627451]
[ 0.82659356  0.83200174  0.89928315  0.77346437]
[ 0.76583439  0.917481    0.88795968]
[ 0.54235493  0.62721295  0.84265166  0.77102409  0.76931761  0.56722689]
[ 0.66666667  0.

[ 0.90551469  0.90074357]
[ 0.53834116  0.4707271   0.77536646  0.3833581   0.89985302  0.81458495
  0.92924177  0.7696      0.82350629  0.78792822]
[ 0.87002505  0.61872288  0.66731518  0.88135717  0.78904429]
[ 0.          0.85647376]
[ 0.48193305  0.8028991   0.92250712  0.61811089]
[ 0.88206398  0.86235326  0.80927712  0.76782822  0.60428987]
[ 0.          0.87382893  0.          0.          0.        ]
[ 0.9194526   0.52397923  0.2961022   0.00352003  0.24141106]
[ 0.          0.91005282  0.77433462  0.84152255]
[ 0.74106324  0.81394479]
[ 0.84113475  0.85158767  0.68345324]
[ 0.85690516  0.89058524  0.47520661  0.38762887  0.89383562]
[ 0.7521616   0.57396136]
[ 0.89590871  0.87760703  0.8593435   0.86872524  0.67667292  0.67283163
  0.64041514  0.64444444  0.6899652   0.68612079]
[ 0.70799104  0.7004519   0.89287808  0.85196791  0.68119748]
[ 0.81950121  0.9294342 ]
[ 0.95888771  0.8836478 ]
[ 0.75199169  0.91602791  0.79255542  0.71060456]
[ 0.43538136  0.72329142  0.91308523]


[ 0.94480551  0.6901888   0.85620374  0.53770919  0.90544321  0.78346457]
[ 0.8819024   0.94838599  0.90734234  0.74600773  0.89109182  0.77207763
  0.83143996  0.7574281 ]
[ 0.77490329  0.91717879  0.03335974]
[ 0.62913415  0.65827762  0.45404412  0.69687109  0.77976025]
[ 0.72259949  0.          0.69445747]
[ 0.85273492  0.86658688  0.61022928  0.          0.        ]
[ 0.09787234  0.86361057  0.87909447  0.87624193  0.93710366  0.87141467
  0.54484189  0.21173763]
[ 0.84301413  0.77737973]
[ 0.3737929  0.       ]
[ 0.87283844  0.          0.54428874]
[ 0.86470129  0.        ]
[ 0.85985968  0.85831205  0.97323119  0.9531635   0.          0.
  0.80963353  0.88506429]
[ 0.          0.95958109  0.39615129  0.73197674]
[ 0.          0.95183135  0.49219949  0.86599692  0.7541424   0.64795642]
[ 0.71254281  0.94082678  0.81366751  0.77379553]
[ 0.93317193  0.89524261]
[ 0.82330591  0.91387706  0.61803813]
[ 0.91349028  0.88964483  0.94748968  0.76043768  0.91367193  0.93096989]
[ 0.4416788

In [33]:
print(geonet_mean_ov)
print(counter)
print(geonet_mean_ov/counter)

7695.32689213
11455
0.67178759425


In [34]:
####################################################################

In [35]:
print(len(imgovlist))
print(len(geonet_part_all))

3474
3474


In [36]:
imglist = fcis_list[:]
orderidx = np.argsort(imgovlist)
orderidx = orderidx[::-1]
or_imglist = []
for x in orderidx:
        print(imgovlist[x])
        or_imglist.append(imglist[x])
    ################################## 

0.95609444536
0.95303878409
0.949853171517
0.948022436459
0.947427151226
0.946105273665
0.945370638421
0.94524424757
0.943372707307
0.940775411937
0.939266742332
0.935984416273
0.935755171421
0.935665457855
0.935027694733
0.934644297786
0.932776357664
0.932772270607
0.93260185062
0.930314002493
0.929854520022
0.92791162419
0.927771582494
0.926039041816
0.925983268076
0.925915090032
0.924913077689
0.924046505298
0.923429903646
0.923002926155
0.921267752484
0.92111355833
0.920926424143
0.920712578598
0.920534261708
0.919235470374
0.914626178789
0.914243653169
0.914207274531
0.912971363682
0.912689849034
0.912645950046
0.912254084684
0.911547408768
0.911302814825
0.910992679592
0.910290618218
0.910097527684
0.909193365243
0.908258222779
0.907853864375
0.906948435556
0.906518425887
0.90587072897
0.90503122554
0.904837004828
0.90434985945
0.904137026038
0.903783567172
0.903758445861
0.903446278005
0.90312912899
0.902432390216
0.902308422173
0.902215387483
0.901577676065
0.901548671594
0.901

0.733643748116
0.733599105515
0.733294720465
0.732669154039
0.731947551382
0.731624021875
0.731030433
0.731029763995
0.730755414181
0.730130100559
0.729870464425
0.729559831478
0.729286030196
0.729221932708
0.728855986244
0.728846918968
0.727971719962
0.727371223982
0.727112990096
0.726801350143
0.726727882344
0.726546203283
0.726523811604
0.726082837756
0.72489213563
0.724400517037
0.7241752134
0.722876255168
0.722842482288
0.722477768805
0.722444850885
0.722378338818
0.722299148173
0.721994801911
0.721926784874
0.720606451729
0.720066751412
0.719357451971
0.719296514986
0.719250706002
0.718924417447
0.718455379875
0.717804555185
0.716975007672
0.71663159441
0.716451514916
0.715946810555
0.715900117698
0.715517368263
0.71462253715
0.713826955403
0.713641152437
0.713626322988
0.713515456151
0.713356780389
0.713002535244
0.712930777646
0.712757296265
0.712554856267
0.712387874649
0.712056244949
0.711837337142
0.711284941907
0.710638708244
0.710545895687
0.710539260217
0.710334870704
0.7

0.485866083994
0.485361736027
0.483691036693
0.482997078131
0.482287847343
0.481851884399
0.481414615177
0.48123177292
0.480647954874
0.480535204222
0.479733272553
0.479535586975
0.479435746297
0.479408657942
0.479200633687
0.478385151772
0.477787050566
0.47650223866
0.476316141314
0.47621371713
0.475732658448
0.475211605931
0.474893897206
0.474771042812
0.4746182295
0.473476775877
0.473334615021
0.472907921928
0.472874289061
0.472598088036
0.472375724621
0.47235231852
0.472304308829
0.472186937431
0.471470771007
0.471071442097
0.470401685094
0.470250255363
0.469707469446
0.469351644366
0.469158060352
0.468887972801
0.46887643243
0.467221789064
0.466786431932
0.466281315903
0.465910215339
0.464884957911
0.46486851217
0.464436599042
0.462127906397
0.461702190463
0.461638310065
0.461555474746
0.461483862651
0.460350754566
0.460349621201
0.460312746899
0.459947809302
0.459592127628
0.459316673934
0.45867216558
0.457936974995
0.457696291645
0.457363881725
0.456843865447
0.456744316534
0.45

In [40]:
import shutil

folderpath = 'Exper_FCIS_DCN/'

for img in or_imglist[0:100]:
    ourmask = img[:img.index('.')] +'_mask.png'
    fcismask = img
    oriimg =  img[:img.index('.')] +'.jpg'
    gtimg =  img
    
    if os.path.exists(folderpath+'10_30_FCIS_DCN/'  + ourmask):  
        shutil.copyfile(folderpath+'10_30_FCIS_DCN/'  + ourmask,folderpath+'compbnf/ours/'+ourmask)
        shutil.copyfile(folderpath+'FCIS_baseline/' +  fcismask,folderpath+'compbnf/baseline/'+fcismask)
        shutil.copyfile(folderpath+'img/' + oriimg,folderpath+'compbnf/img/'+oriimg)
        shutil.copyfile(folderpath+'gt/' + gtimg,folderpath+'compbnf/gt/'+gtimg)
    